# Shopping Flag and Probability Model Update（After Feature Selection）

### Logistic Regression Model

### import data

In [1]:
import pandas as pd
import numpy as np

In [2]:
Customer_L = pd.read_csv('data.csv')
Customer_L.head()

,CUSTOMER_DIM_ID,LIFETIME_UNITS,DISTINCT_CHANNELS_PURCHASED,DIRECT_LIFETIME_UNITS,TOT_ORD_365_POST_ACQ,TOT_ORD_365_SECOND_PLUS,TOT_UNITS_365_SECOND_PLUS,TOT_UNITS_365_POST_SEC,LAPSED_CUSTOMER_FLG,Lifetime_Womens_Purchase_Flg,next_12mo_buyer_flg
0,8376,1,1,1,0,0,0,0,1,1,0
1,9729,51,1,50,1,6,19,12,0,1,1
2,12765,4,2,4,1,1,2,0,1,1,0
3,14940,1,1,1,0,0,0,0,1,0,0
4,15046,1,1,1,0,0,0,0,1,1,0


In [3]:
Customer_L.set_index('CUSTOMER_DIM_ID', inplace=True)
Customer_L.head()

,LIFETIME_UNITS,DISTINCT_CHANNELS_PURCHASED,DIRECT_LIFETIME_UNITS,TOT_ORD_365_POST_ACQ,TOT_ORD_365_SECOND_PLUS,TOT_UNITS_365_SECOND_PLUS,TOT_UNITS_365_POST_SEC,LAPSED_CUSTOMER_FLG,Lifetime_Womens_Purchase_Flg,next_12mo_buyer_flg
CUSTOMER_DIM_ID,,,,,,,,,,
8376,1,1,1,0,0,0,0,1,1,0
9729,51,1,50,1,6,19,12,0,1,1
12765,4,2,4,1,1,2,0,1,1,0
14940,1,1,1,0,0,0,0,1,0,0
15046,1,1,1,0,0,0,0,1,1,0


### Divide Data

In [9]:
Customer_L=pd.DataFrame(Customer_L)
y_L=Customer_L['next_12mo_buyer_flg']
X_L=Customer_L.drop('next_12mo_buyer_flg', axis=1)
X_L.head

<bound method NDFrame.head of                  LIFETIME_UNITS  DISTINCT_CHANNELS_PURCHASED  \
CUSTOMER_DIM_ID                                                
8376                          1                            1   
9729                         51                            1   
12765                         4                            2   
14940                         1                            1   
15046                         1                            1   
18536                        77                            1   
22137                        60                            1   
31008                        33                            2   
34618                         1                            1   
37727                         5                            1   
39569                         1                            1   
40044                        12                            1   
44927                        20                            1   
47859     

### Train and Test Data

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train_L, X_test_L, y_train_L, y_test_L = train_test_split(X_L, y_L, test_size=0.2, random_state = 42)
logreg = LogisticRegression()
logreg.fit(X_train_L, y_train_L)
y_pred_L = logreg.predict(X_test_L)
y_pred_proba_L = logreg.predict_proba(X_test_L)

In [6]:

X_test_L=pd.DataFrame(X_test_L)
result_L=pd.DataFrame(y_pred_proba_L)
result_L.columns = ['flag = 0', 'flag = 1']
result_L.index=X_test_L.index
df_out = pd.concat([X_test_L,result_L], axis=1)
df_out=df_out.sort_values(by='flag = 1')

In [7]:
df_out.head()

,LIFETIME_UNITS,DISTINCT_CHANNELS_PURCHASED,DIRECT_LIFETIME_UNITS,TOT_ORD_365_POST_ACQ,TOT_ORD_365_SECOND_PLUS,TOT_UNITS_365_SECOND_PLUS,TOT_UNITS_365_POST_SEC,LAPSED_CUSTOMER_FLG,Lifetime_Womens_Purchase_Flg,flag = 0,flag = 1
CUSTOMER_DIM_ID,,,,,,,,,,,
785095,232,2,231,11,13,76,74,1,1,0.998853,0.001147
933224,26,1,26,0,4,25,24,1,0,0.997109,0.002891
877303,62,2,59,6,6,32,30,1,1,0.994452,0.005548
2988565,1,1,1,0,0,0,0,1,1,0.994408,0.005592
39569,1,1,1,0,0,0,0,1,1,0.994408,0.005592


In [8]:
df_out.to_csv("Logistics_predictions_Update.csv")

In [ ]:
logreg.score(X_test_L, y_test_L)

### Validate Data

#### ROC Curve - Threshold

In [10]:
from sklearn.metrics import roc_curve
y_pred_proba_L = logreg.predict_proba(X_test_L)[:,1]
fpr, tpr, thresholds = roc_curve(y_test_L, y_pred_proba_L)

In [11]:
import matplotlib.pyplot as plt
plt.plot([0,1], [0,1], 'k--')
plt.plot(fpr, tpr, label = 'Logistic Regression')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression ROC Curve')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test_L, y_pred_proba_L)
# Larger area under ROC curve = better model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test_L, y_pred_L))
print(classification_report(y_test_L, y_pred_L))

# precision = TP/(TP+FP)
# recall=TP/(TP+FN)


# precision - fraction of retrieved instances that are relevant
# recall - fraction of relevant instances that are retrieved
# F1 score - measure of a test's accuracy
# support - count of testes samples